In [2]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


[notice] A new release of pip is available: 23.1.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     --------------------------------------- 17.7/17.7 MB 26.1 MB/s eta 0:00:00


In [5]:
import pulp

# TODO: zaimplementować rozwiązanie problemu UTA-GMS za pomocą biblioteki PuLP
# to tylko szkielet, niekoniecznie te pętle i warunki są prawidłowo zaimplementowane

model = pulp.LpProblem("UTA_GMS_Problem", pulp.LpMaximize)

num_criteria = 4
num_variants = 4

# non-negativity
u_best = [pulp.LpVariable(f'u_best_{i}', lowBound=0) for i in range(num_criteria)]
u_worst = [pulp.LpVariable(f'u_worst_{i}', lowBound=0, upBound=0) for i in range(num_criteria)]

epsilon = pulp.LpVariable('epsilon', lowBound=0)

u_variants = [[pulp.LpVariable(f'u_variant_{i}_{j}') for j in range(num_criteria)] for i in range(num_variants)]

preferential_information = [(0, 1), (1, 2), (2, 3)]

# rankingi preferencyjne
for (a, b) in preferential_information: 
    model += pulp.lpSum(u_variants[a][j] for j in range(num_criteria)) >= pulp.lpSum(u_variants[b][j] for j in range(num_criteria)) + epsilon

for i in range(num_criteria):
    # normalizacja
    model += u_best[i] == 1
    model += u_worst[i] == 0

    # monotoniczność
    for k in range(num_variants - 1):
        model += u_variants[k+1][i] >= u_variants[k][i]

model += epsilon


model.solve()

for v in model.variables():
    print(v.name, "=", v.varValue)

print("Status:", pulp.LpStatus[model.status])


epsilon = 0.0
u_best_0 = 1.0
u_best_1 = 1.0
u_best_2 = 1.0
u_best_3 = 1.0
u_variant_0_0 = -0.0
u_variant_0_1 = 0.0
u_variant_0_2 = 0.0
u_variant_0_3 = -0.0
u_variant_1_0 = 0.0
u_variant_1_1 = 0.0
u_variant_1_2 = 0.0
u_variant_1_3 = 0.0
u_variant_2_0 = 0.0
u_variant_2_1 = 0.0
u_variant_2_2 = 0.0
u_variant_2_3 = 0.0
u_variant_3_0 = -0.0
u_variant_3_1 = -0.0
u_variant_3_2 = 0.0
u_variant_3_3 = 0.0
u_worst_0 = 0.0
u_worst_1 = 0.0
u_worst_2 = 0.0
u_worst_3 = 0.0
Status: Optimal
